In [78]:
from datetime import date, datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import inflation

"""
Quais foram as compras de jogadores com melhores e 
piores custo-benefício registradas?
"""

#Função para abrir arquivos csv
def open_csv(arquivo : str) -> pd.core.frame.DataFrame:
    with open('../data/'+arquivo+'.csv', 'r', encoding='latin1') as file:
        return pd.read_csv(file)

#Função para calcular custo-beneficio
def calc_cost_benefit(group : pd.core.frame.DataFrame) -> float:
    delta_price = (group["market_value_in_eur_shift"].iloc[0] - group["market_value_in_eur"].iloc[0])
    modificador = 10 ** (len(str(delta_price))-1)
    #Fórmula para calcular desempenho do jogador, segundo sites esportivos (contem modificacao)
    reduce = sum((-1)*group["yellow_cards"] + (-3)*group["red_cards"] + (8)*group["goals"] + (5)*group["assists"]) * modificador / group.shape[0]
    return round((reduce + delta_price) / group["market_value_in_eur"].iloc[0], 4)
    
#Função correção da data_diff
def correct_data_shift(row : pd.core.series.Series) -> datetime.date:
    if not row["same_player"]:
        return datetime.now().date()
    return row["date_shift"].date()

#Função correção da market_value_in_eur_shift
def correct_market_value_in_eur_shift(row : pd.core.series.Series) -> float:
    if not row["same_player"]:
        return row["current_market_value"]
    return row["market_value_in_eur_shift"]

#Abrindo as tabelas que serão utilizadas
appearances = open_csv("appearances")
players = open_csv("players")
transfers = open_csv("transfers")

#Limpando dados NaN das colunas que serão utilizadas
appearances.dropna(axis=0, subset=["yellow_cards", "red_cards", "goals", "assists"], inplace=True) 
transfers.dropna(axis=0, subset=["player_name", "transfer_date","market_value_in_eur", "from_club_id", "to_club_id"], inplace=True)
players.dropna(axis=0, subset=["name", "market_value_in_eur"], inplace=True)

#Criando coluna same_player
transfers.sort_values(['player_id', 'transfer_date'], ascending=True, inplace=True)
transfers["transfer_date"] = pd.to_datetime(transfers["transfer_date"], yearfirst=True)
transfers["same_player"] = -transfers["player_id"].diff(-1) == 0

#Criando coluna data_shift
transfers['date_shift'] = transfers['transfer_date'].shift(-1)
transfers["date_shift"] = transfers.apply(correct_data_shift, axis=1)

#Criando coluna market_value_in_eur_shift
transfers['market_value_in_eur'] = inflation.inflation_adj(transfers['market_value_in_eur'], transfers["transfer_date"])
transfers['market_value_in_eur_shift'] = transfers['market_value_in_eur'].shift(-1)
players.rename(columns={'market_value_in_eur': 'current_market_value'}, inplace=True)
transfers = pd.merge(players[["player_id", "current_market_value"]], transfers, on='player_id')
transfers["market_value_in_eur_shift"] = transfers.apply(correct_market_value_in_eur_shift, axis=1)

#Criando performance
appearances.sort_values(['player_id', 'date'], ascending=True, inplace=True)
appearances["date"] = pd.to_datetime(appearances["date"], yearfirst=True)

#Unindo as tabelas
merged = pd.merge(appearances, transfers, on='player_id')
merged = merged[(merged['date'] >= merged['transfer_date']) & (merged['date'] <= merged['date_shift'])]
merged.rename(columns={'player_name_x': 'player_name'}, inplace=True)

#Agrupar por transferência
cost_benefit = merged.groupby(['player_id', 'player_name', 'transfer_date', 'from_club_id', 'from_club_name', 'to_club_id', 'to_club_name']).apply(calc_cost_benefit).reset_index(name="cost_benefit")
cost_benefit.sort_values(by='cost_benefit', ascending=False, inplace=True)
cost_benefit

#Plotando o gráfico
print(cost_benefit[["cost_benefit"]].describe())
sns.boxplot(data=cost_benefit,  y="cost_benefit", color="red")
plt.show()



C:\Users\luizb\AppData\Local\Temp\ipykernel_17352\443129946.py:71: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cost_benefit = merged.groupby(['player_id', 'player_name_x', 'transfer_date', 'from_club_id', 'from_club_name', 'to_club_id', 'to_club_name']).apply(calc_cost_benefit).reset_index(name="cost_benefit")


,player_id,player_name_x,transfer_date,from_club_id,from_club_name,to_club_id,to_club_name,cost_benefit
11922,342229,Kylian MbappÃ©,2016-01-01,9669,Monaco U19,162,Monaco,95801.0408
16704,533738,Jonathan David,2018-07-01,43562,KAA Gent U21,157,KAA Gent,49204.2239
13406,386726,Yusuf YazÄ±cÄ±,2015-12-21,10446,Trabzonspor U21,449,Trabzonspor,48534.8621
15812,485424,Moussa Djenepo,2017-07-01,60309,Yeelen,3057,Standard LiÃ¨ge,47858.0476
5125,167850,Memphis Depay,2012-01-01,4575,PSV U19,383,PSV Eindhoven,43287.8496
...,...,...,...,...,...,...,...,...
12420,355501,Amadou Diawara,2015-07-01,10155,San Marino,1025,Bologna,-759.2143
14618,429901,Paul Akouokou,2018-08-21,22890,EkenÃ¤s IF,2865,Betis Deportivo,-861.0000
14388,417913,Ãlvaro Valles,2019-07-01,11610,Las Palmas Atl.,472,UD Las Palmas,-1232.3514
5527,177843,Domenico Berardi,2012-07-01,19114,Sassuolo U19,6574,Sassuolo,-1254.3333
